# <strong> Coding part of the Database Project </strong>

### <strong> Brief </strong>

<p> <em> In this project we are going to design an Relational database model (E/R) for </em> </p>
<p> <em> an Software Project Management System. This project are going to consist of a Report, </em> </p>
<p> <em> SQL code, and Design of a Relational Database Model. </em> </p>

#### <strong> Sprint 1 </strong>
* <em> Design a relational database (E/R diagram) for a software project management system </em>
* <em> Fullfills the given requirments </em>

#### <strong> Sprint 2 </strong>
* <em> Write down SQL for the given queires </em>
* <em> Write the SQL based on the designed database </em>

#### <strong> Sprint 3 </strong>
* <em> Start report "Conceptual Design for Library Management/Project Management Database" </em>
* <em> Write Problem Analysis </em>
* <em> Write Conceptual Design (E/R diagram and description) </em>
* <em> Explain the SQL code </em>

In [12]:
%load_ext sql
%sql sqlite:///project_management_system.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @project_management_system.db'

#### Creating table Project

In [13]:
%%sql
DROP TABLE IF EXISTS Project;
CREATE TABLE Project (
    projectID int NOT NULL UNIQUE,
    name varchar(150) NOT NULL DEFAULT '',
    startDate date,
    endDate date,
    leader varchar(150) NOT NULL DEFAULT '',
    budget int,
    PRIMARY KEY (projectID)
);

 * sqlite:///project_management_system.db
Done.
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

#### Creating the table Employee

In [14]:
%%sql
DROP TABLE IF EXISTS Employee;
CREATE TABLE Employee (
    employeeID int NOT NULL UNIQUE,
    name varchar(150) NOT NULL DEFAULT '',
    hourlyCost int NOT NULL DEFAULT '',
    PRIMARY KEY (employeeID)
);

 * sqlite:///project_management_system.db
Done.
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

#### Creating the table Activity

In [21]:
%%sql
DROP TABLE IF EXISTS Activity;
CREATE TABLE Activity (
    activityID int NOT NULL UNIQUE,
    activityType varchar(150) NOT NULL DEFAULT '',
    startDate date,
    endDate date,
    PRIMARY KEY (activityID)
);

 * sqlite:///project_management_system.db
Done.
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

#### Checking that Activity does not use an invalid Date

In [22]:
%%sql
CREATE TRIGGER activity_date_check
BEFORE INSERT ON Activity
WHEN NEW.startDate < (SELECT startDate FROM Plan WHERE activityID = NEW.activityID)
OR NEW.startDate > (SELECT endDate FROM Plan WHERE activityID = NEW.activityID)
OR NEW.endDate < (SELECT startDate FROM Plan WHERE activityID = NEW.activityID)
OR NEW.endDate > (SELECT endDate FROM Plan WHERE activityID = NEW.activityID)
OR NEW.startDate NOT BETWEEN (SELECT startDate FROM Activity WHERE activityID = NEW.activityID) AND (SELECT endDate FROM Activity WHERE activityID = NEW activityID)
OR NEW.endDate NOT BETWEEN (SELECT startDate FROM Activity WHERE activityID = NEW.activityID) AND (SELECT endDate FROM Activity WHERE activityID = NEW activityID)
BEGIN
SELECT RAISE(FAIL, "invalid date in activity");
END;

 * sqlite:///project_management_system.db
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

#### Creating the table for Plan

In [24]:
%%sql
DROP TABLE IF EXISTS Plan;
CREATE TABLE Plan (
    planName varchar(150) NOT NULL,
    planID int NOT NULL UNIQUE,
    projectID int NOT NULL,
    employeeID int NOT NULL,
    activityID int NOT NULL,
    startDate date,
    endDate date,
    budget int,
    PRIMARY KEY (PlanID),
    FOREIGN KEY (projectID) REFERENCES Project(projectID),
    FOREIGN KEY (employeeID) REFERENCES Employee(employeeID),
    FOREIGN KEY (activityID) REFERENCES Activity(activityID)
);

 * sqlite:///project_management_system.db
Done.
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

#### Check that Plan does not use an invalid date

In [25]:
%%sql
CREATE TRIGGER Plan_date_check
BEFORE INSERT ON Plan
WHEN NEW.startDate < (SELECT startDate FROM Project WHERE planID = ID)
OR NEW.startDate > (SELECT endDate FROM Project WHERE planID = NEW.planID)
OR NEW.endDate < (SELECT startDate FROM Project WHERE planID = NEW.planID)
OR NEW.endDate > (SELECT endDate FROM Project WHERE planID = NEW.planID)
BEGIN
SELECT RAISE(FAIL, "invalid date in plan");
END;

 * sqlite:///project_management_system.db
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

#### Gives Employees access to Project, Activity and Plans tables

In [27]:
%%sql
GRANT SELECT
On TABLE Project, Activity, Plans 
TO Employee;

UsageError: Cell magic `%%mysql` not found.


#### How many employees of for a project titled "A" are involved in its plan “B”

In [29]:
%%sql
SELECT COUNT(employeeID)
FROM Employee
INNER JOIN Plan USING(employeeID)
INNER JOIN Project USING(projectID)
WHERE Project.name = "A"
AND Plan.planName = "B";

 * sqlite:///project_management_system.db
Done.


COUNT(employeeID)
0


#### Retrieve the names of plans made for project “A” with least cost

In [31]:
%%sql
SELECT name
FROM Plan
INNER JOIN Project USING(projectID)
WHERE Project.name = "A"
AND Project.budget IN(
	SELECT MIN(budget)
	FROM Project
);

 * sqlite:///project_management_system.db
Done.


name


#### For each employee retrieve the name, project name and plan name with the most working time

In [44]:
%%sql
SELECT Employee.name, Project.name, Plan.planID
FROM Employee
INNER JOIN Plan USING(employeeID)
INNER JOIN Project USING(projectID)
GROUP BY Employee.name
HAVING (Plan.endDate-PLan.startDate) = (
	SELECT MAX(endDate-startDate)
	FROM Plan);

 * sqlite:///project_management_system.db
Done.


name,name_1,planID


#### Retrieve all the employee’s name and their least working time with respect to different project

In [46]:
%%sql
SELECT Employee.name, (Project.endDate-Project.startDate) as leastTime
FROM Employee
INNER JOIN Plan USING (employeeID)
INNER JOIN Project USING(projectID)
GROUP BY Employee.name
HAVING leastTime = (
    SELECT MIN(leastTime) FROM Project
);

 * sqlite:///project_management_system.db
Done.


name,leastTime


#### Retrieve all the plans for project with order of their working period

In [47]:
%%sql
SELECT *
FROM Plan
GROUP BY projectID
ORDER BY startDate;

 * sqlite:///project_management_system.db
Done.


planName,planID,projectID,employeeID,activityID,startDate,endDate,budget
